In [2]:
from bs4 import BeautifulSoup
import urllib.parse as parse
from urllib.request import urlopen
import time

import os

import html.parser
import re

import unicodedata

from selenium import webdriver
from selenium.webdriver.support import ui

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains




import requests


In [3]:
class Browser:
    def __init__(self):
        self.options= webdriver.FirefoxOptions()
        self.options.binary= "/usr/bin/firefox"
        self.options.add_argument('--headless')
        
        self.profile= webdriver.FirefoxProfile()
        self.profile.set_preference("geo.prompt.testing", True);
        self.profile.set_preference("geo.prompt.testing.allow", True);
        self.profile.set_preference("geo.enabled", True);
        self.profile.set_preference("dom.webdriver.enabled", False);
        self.profile.set_preference("geo.wifi.uri", 
                          'data:application/json,{\"location\": {\"lat\": 40.71, \"lng\": 70.00}, \"accuracy\": 100.0}');
        
        self.browser = webdriver.Firefox(
        executable_path="/home/jatinkathuria/projects/indeedResume/firefox/geckodriver",
        options=self.options,
        firefox_profile= self.profile)
    
    def getNextPageSource
    def getResultPageSource(self, url,initial=True):
        self.browser.get(url)
        try:
            if(initial == True):                
                element= WebDriverWait(self.browser, 100,poll_frequency=1).until(
                    EC.presence_of_element_located((By.ID,'footerLocale'))
                )
                print(element)
                html= self.browser.page_source
        #         print(html)
                self.browser.save_screenshot('./headless_firefox_test_01.png')        
                current_selection= ui.Select(self.browser.find_element_by_id('footerLocale'))
                time.sleep(20)
                current_selection.select_by_value('US')
                self.browser.find_element_by_class_name('icl-WhatWhere-button').click()
            wait = WebDriverWait(self.browser, 100)
            wait.until(
                EC.presence_of_element_located((By.CLASS_NAME,'rezemp-ResumeSearchCard'))
            );
            self.browser.save_screenshot('./headless_firefox_test.png');
            ### Result are now successfully displayed.
            soup = BeautifulSoup(self.browser.page_source,'html.parser')
            
            return soup
        except:
            self.browser.save_screenshot('./headless_firefox_test_except.png')
    #         print(browser.page_source)
            raise;
        


In [4]:
def get_cv_source(id_='', url=''):
#     id_='f44472876d41589f'
    if(len(url) == 0):
        prefix= 'https://resumes.indeed.com/resume/'
        url = prefix+id_
    job_browser = Browser()
    job_browser_handle = job_browser.browser
    job_browser_handle.get(url)
    try:
        element= WebDriverWait(job_browser_handle, 100,poll_frequency=2).until(
            EC.presence_of_element_located((By.ID,'resume_body'))
        )
        return job_browser_handle.page_source
    except:
        job_browser_handle.save_screenshot('./get_cv_source_except.png')
        raise;

def get_basic_info(source,result={}):
#     print(source)
    resume_contact_tag = source.findChildren('div',{'id':'resume-contact'})
    resume_headline_tag = source.findChildren('div',{'id':'headline'})
    resume_address_tag = source.findChildren('div',{'id':'contact_info_container'})
    
    
    result['contact']= resume_contact_tag[0].get_text() if len(resume_contact_tag) > 0 else ''
    result['headline']= resume_headline_tag[0].get_text() if len(resume_headline_tag) > 0 else ''
    result['addr']= resume_address_tag[0].get_text() if len(resume_address_tag) > 0 else ''

def get_work_exp(source, result= {}):
    result['workexp'] = source.get_text()
   

def get_education_data(source, result={}):
    result['education'] = source.get_text()
    

def get_skills_content(source, result={}):
    result['skills'] = source.get_text()

def get_add_info(source, result={}):
    result['additional'] = source.get_text()

def extractCVFromId(id='', sections= {}):
    jd_src = get_cv_source(id);
    
    sections = {
    'basicInfo-content': get_basic_info,
    'workExperience-content':get_work_exp,
    'education-content': get_education_data,
    'skills-content' : get_skills_content,
    'additionalInfo-content': get_add_info
    }
    jd_src_soup = BeautifulSoup(jd_src,'html.parser')
    unwanted = jd_src_soup.find_all(re.compile(r'display: none'))
    for items in unwanted:
        item.extract()

    main_area = jd_src_soup.find_all(id='resume_body')[0];
    result = {}
    for sec in sections.keys():
#         print('Now running for ',  sec)
        new_section = main_area.findChildren(class_=sec)
        if(len(new_section) > 0 ):
            sections[sec](main_area.findChildren(class_=sec)[0],result)
    return result

# extractCVFromId()  

In [ ]:
def scrape(query, number_of_results=50, output_dir='./CVs', continue_from=1):
    query_dir = os.path.join(output_dir, query)
    no_of_pages = int(number_of_results/50)
    
#     time.sleep(5)
#     print('Looking for page : ', page)
    params = {
        "q": query,
        "search_fields": ','.join(['jt','skills','fos']),
    }

    initial = True

    base_url="https://resumes.indeed.com/search"

    res = requests.get(url=base_url, 
                       params=params,
                     )
    full_url = res.url
    print('URL is : ', full_url)
    result_browser = Browser();

    source = result_browser.getResultPageSource(full_url, initial)
    print('Got the Search results. Saving the Screenshot')
    for page in range(no_of_pages):
        if not os.path.exists(query_dir):
            os.makedirs(query_dir)
        result_browser.browser.save_screenshot(os.path.join(query_dir, 'search_result_'+str(page+1)+'.png'));

        links = source.find_all("a",class_='icl-TextLink--primary')
        # print(links)
        id_links = [full_link['href'].split('?')[0].split('/')[2] for full_link in links]
        print("Got the Links to the page result")
        # print('.' in id_links[0])
        print('Starting to write CVs')
        for id in id_links:
            cv_path = os.path.join(query_dir, id +'.txt')
            if(os.path.isfile(cv_path)):
                print("Already saved CV for Id : ", cv_path)
                pass
            else:
                print("saving CV for Id : ", id)
                if '.' in id:
                    pass
                else:
                    cv = extractCVFromId(id)
                    with open(cv_path,'w') as f:
                        f.write(str(cv));
        print("Navigative to Next page")
        result_browser.browser.save_screenshot('./before_next.jpg')
        try:
            result_browser.browser.find_element_by_class_name('rezemp-pagination-nextbutton').click()
            source = BeautifulSoup(result_browser.browser.page_source,'html.parser')
        except NoSuchElementException:
            print('Got all the results')
        result_browser.browser.save_screenshot('./after_next.png')
scrape('Order to cash',number_of_results=1000)


URL is :  https://resumes.indeed.com/search?q=Order+to+cash&search_fields=jt%2Cskills%2Cfos
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="eb9bf423-cfc9-4bc8-a341-bf158856249e", element="e561bba1-83f8-44b2-b8ec-de3913b32f16")>
Got the Search results. Saving the Screenshot
Got the Links to the page result
Starting to write CVs
Already saved CV for Id :  ./CVs/Order to cash/7732baeeac1fd38d.txt
Already saved CV for Id :  ./CVs/Order to cash/ad6172e462222616.txt
Already saved CV for Id :  ./CVs/Order to cash/e9bba40e5ceab619.txt
Already saved CV for Id :  ./CVs/Order to cash/5085bfd7c0bb05a4.txt
Already saved CV for Id :  ./CVs/Order to cash/aae54e2e151ce0b6.txt
Already saved CV for Id :  ./CVs/Order to cash/e604a3b45c6daf01.txt
Already saved CV for Id :  ./CVs/Order to cash/227948b3c6037fc3.txt
Already saved CV for Id :  ./CVs/Order to cash/40cea416386ac6b7.txt
Already saved CV for Id :  ./CVs/Order to cash/d05b9af936bdc2c2.txt
Already saved CV for Id :  ./CVs/Order t

Got the Links to the page result
Starting to write CVs
Already saved CV for Id :  ./CVs/Order to cash/4156c3149472fd93.txt
Already saved CV for Id :  ./CVs/Order to cash/a9bc80c95c1bf262.txt
Already saved CV for Id :  ./CVs/Order to cash/7d5464a267a83dac.txt
Already saved CV for Id :  ./CVs/Order to cash/f45de766cdc8195f.txt
Already saved CV for Id :  ./CVs/Order to cash/95bf2ec98856c107.txt
Already saved CV for Id :  ./CVs/Order to cash/af7413d2cb00d593.txt
Already saved CV for Id :  ./CVs/Order to cash/2495b91fd396b47c.txt
Already saved CV for Id :  ./CVs/Order to cash/94aa99887b184fe4.txt
Already saved CV for Id :  ./CVs/Order to cash/6d6bfb17a6730044.txt
Already saved CV for Id :  ./CVs/Order to cash/bb946b821fe6bf1c.txt
Already saved CV for Id :  ./CVs/Order to cash/a3a5ab5b8cc868fc.txt
Already saved CV for Id :  ./CVs/Order to cash/54262bbeba57b84d.txt
Already saved CV for Id :  ./CVs/Order to cash/d574b79df98a55ec.txt
Already saved CV for Id :  ./CVs/Order to cash/8030965c062965